# PaddleHub实战——使用预训练模型进行多标签文本分类

**百度试题知识点标注数据集**
本项目代码需要使用GPU环境来运行:

本项目将演示，如何使用PaddleHub语义预训练模型ERNIE对自定义数据集完成文本分类。

重要参考: 
- [PaddleHub 多标签分类](https://github.com/PaddlePaddle/PaddleHub/tree/release/v1.6/demo/multi_label_classification) 
- [paddle 多分类](https://aistudio.baidu.com/aistudio/projectdetail/221999)
- [PaddleHub自定义数据集](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub%E9%80%82%E9%85%8D%E8%87%AA%E5%AE%9A%E4%B9%89%E6%95%B0%E6%8D%AE%E5%AE%8C%E6%88%90FineTune)

# 数据说明
原始数据集为`高中`下`地理`,`历史`,`生物`,`政治`四门学科数据，每个学科下各包含第一层知识点，如`历史`下分为`近代史`,`现代史`,`古代史`。  
原始数据示例： 

> [题目]  
我国经济体制改革首先在农村展开。率先实行包产到组、包产到户的农业生产责任制的省份是（    ）  
①四川        ②广东        ③安徽       ④湖北A. ①③B. ①④C. ②④D. ②③题型: 单选题|难度: 简单|使用次数: 0|纠错复制收藏到空间加入选题篮查看答案解析答案：A解析：本题主要考察的是对知识的识记能力，比较容易。根据所学知识可知，在四川和安徽，率先实行包产到组、包产到户的农业生产责任制，故①③正确；②④不是。所以答案选A。知识点：  
[知识点：]  
经济体制改革,中国的振兴

对数据处理：
- 将数据的[知识点：]作为数据的第四层标签，显然不同数据的第四层标签数量不一致
- 仅保留题目作为数据特征，删除[题型]及[答案解析]

# Step1: 加载预训练模型
<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_1.png" hspace='10'/> <br />
</p>


<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_2.png" hspace='10'/> <br />
</p>

很神奇，第二次及以后运行时要加入下面这两句话，是为了创建个文件夹。
```
%cd /home/aistudio/
!mkdir -p .paddlehub/cache
```
如果直接
``` python
import paddlehub as hub
module = hub.Module(name="ernie_tiny")
```
会报错
![](https://ai-studio-static-online.cdn.bcebos.com/9095d984e2474cbd8c1864e55eed4fefd6f110e0bc724b1ca240eb2b97f1b187)


In [1]:
%cd /home/aistudio/
!mkdir -p .paddlehub/cache

/home/aistudio


In [2]:
import paddlehub as hub
module = hub.Module(name="bert_wwm_chinese_L-12_H-768_A-12")
inputs, outputs, program = module.context(trainable=True, max_seq_len=256)

[2020-03-26 19:31:41,661] [    INFO] - Installing bert_wwm_chinese_L-12_H-768_A-12 module
[2020-03-26 19:31:41,694] [    INFO] - Module bert_wwm_chinese_L-12_H-768_A-12 already installed in /home/aistudio/.paddlehub/modules/bert_wwm_chinese_L-12_H-768_A-12
[2020-03-26 19:31:42,416] [    INFO] - Set maximum sequence length of input tensor to 256
[2020-03-26 19:31:42,417] [    INFO] - The shape of input tensor[input_ids] set to [-1, 256, 1]
[2020-03-26 19:31:42,417] [    INFO] - The shape of input tensor[position_ids] set to [-1, 256, 1]
[2020-03-26 19:31:42,418] [    INFO] - The shape of input tensor[segment_ids] set to [-1, 256, 1]
[2020-03-26 19:31:42,418] [    INFO] - The shape of input tensor[input_mask] set to [-1, 256, 1]
[2020-03-26 19:31:42,419] [    INFO] - 199 pretrained paramaters loaded by PaddleHub


其中最大序列长度`max_seq_len`是可以调整的参数，建议值128，根据任务文本长度不同可以调整该值，但最大不超过512。

PaddleHub还提供BERT等模型可供选择, 模型对应的加载示例如下：

| 模型名                         | PaddleHub Module                                             |
| ------------------------------ | ------------------------------------------------------------ |
| ERNIE, Chinese                 | `hub.Module(name='ernie')`                                   |
| ERNIE tiny, Chinese            | `hub.Module(name='ernie_tiny')`                              |
| ERNIE 2.0 Base, English        | `hub.Module(name='ernie_v2_eng_base')`                       |
| ERNIE 2.0 Large, English       | `hub.Module(name='ernie_v2_eng_large')`                      |
| BERT-Base, Uncased             | `hub.Module(name='bert_uncased_L-12_H-768_A-12')`            |
| BERT-Large, Uncased            | `hub.Module(name='bert_uncased_L-24_H-1024_A-16')`           |
| BERT-Base, Cased               | `hub.Module(name='bert_cased_L-12_H-768_A-12')`              |
| BERT-Large, Cased              | `hub.Module(name='bert_cased_L-24_H-1024_A-16')`             |
| BERT-Base, Multilingual Cased  | `hub.Module(nane='bert_multi_cased_L-12_H-768_A-12')`        |
| BERT-Base, Chinese             | `hub.Module(name='bert_chinese_L-12_H-768_A-12')`            |
| BERT-wwm, Chinese              | `hub.Module(name='bert_wwm_chinese_L-12_H-768_A-12')`        |
| BERT-wwm-ext, Chinese          | `hub.Module(name='bert_wwm_ext_chinese_L-12_H-768_A-12')`    |
| RoBERTa-wwm-ext, Chinese       | `hub.Module(name='roberta_wwm_ext_chinese_L-12_H-768_A-12')` |
| RoBERTa-wwm-ext-large, Chinese | `hub.Module(name='roberta_wwm_ext_chinese_L-24_H-1024_A-16')` |

更多模型请参考[PaddleHub官网](https://www.paddlepaddle.org.cn/hub)。

# Step2: 准备数据集并使用MultiLabelClassifyReader读取数据


直接从之前处理的bert数据集拿过来。自己新建立一个数据预处理类即可。

其中数据集的准备代码可以参考paddle v1.6的[toxic.py](https://github.com/PaddlePaddle/PaddleHub/blob/release/v1.6/paddlehub/dataset/toxic.py)。

注意以后这个代码可能会过时，记得切换版本。
![](https://ai-studio-static-online.cdn.bcebos.com/7d32862be5e3444eb2e6b4d8fb9081e9f8c9cd06f4fc4190857515d6f2f8e996)

我参考的是paddle v1.2的[toxic.py](https://github.com/PaddlePaddle/PaddleHub/blob/release/v1.2/paddlehub/dataset/toxic.py)。

自定义数据集的处理参考 [自定义数据集](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub%E9%80%82%E9%85%8D%E8%87%AA%E5%AE%9A%E4%B9%89%E6%95%B0%E6%8D%AE%E5%AE%8C%E6%88%90FineTune)

In [1]:
import os
import pandas as pd
import paddlehub as hub
from paddlehub.dataset.dataset import InputExample, BaseDataset

class Baidu(BaseDataset):
    def __init__(self):
        self.dataset_dir = 'work'

        self.label_file = os.path.join(self.dataset_dir, "label.txt")
        self.label_list = [w.strip() for w in open(self.label_file).readlines()]
        self.label_map = {word:idx  for idx, word in enumerate(self.label_list)}

        self._load_train_examples()
        self._load_test_examples()
        self._load_dev_examples()

    # 我自己定义的一个函数
    def labels2index(self, labels, label_map):
        """
        :params: labels 是一串字符串，包含数个标签，标签由空格分开
        """
        index = [0] * len(label_map)
        for label in labels.split():
            index[label_map[label]] = 1
        return index

    def _load_train_examples(self):
        self.train_file = os.path.join(self.dataset_dir, "train.tsv")
        self.train_examples = self._read_csv(self.train_file)

    def _load_dev_examples(self):
        self.dev_file = os.path.join(self.dataset_dir, "dev.tsv")
        self.dev_examples = self._read_csv(self.dev_file)

    def _load_test_examples(self):
        self.test_file = os.path.join(self.dataset_dir, "test.tsv")
        self.test_examples = self._read_csv(self.test_file)

    def get_train_examples(self):
        return self.train_examples

    def get_dev_examples(self):
        return self.dev_examples

    def get_test_examples(self):
        return self.test_examples

    def get_labels(self):
        
        return self.label_list

    @property
    def num_labels(self):
        """
        Return the number of labels in the dataset.
        """
        return len(self.get_labels())

    def _read_csv(self, input_file, quotechar=None):
        """Reads a tab separated value file."""
        data = pd.read_csv(input_file, encoding="UTF-8", sep='\t')
        examples = []
        for index, row in data.iterrows():
            guid = index
            text = row["content"]
            labels = self.labels2index(row['label'], self.label_map)

            example = InputExample(guid=guid, label=labels, text_a=text)
            examples.append(example)

        return examples


[MultiLabelClassifyReader](https://github.com/PaddlePaddle/PaddleHub/blob/45179f3d06ed32ae60a4d8a573829b354548c4b5/paddlehub/reader/nlp_reader.py#L36)的参数：
* `dataset`: 传入PaddleHub Dataset;
* `vocab_path`: 传入ERNIE/BERT模型对应的词表文件路径;
* `max_seq_len`: ERNIE模型的最大序列长度，若序列长度不足，会通过padding方式补到max_seq_len, 若序列长度大于该值，则会以截断方式让序列长度为max_seq_len;
* `sp_model_path`: 传入 ERNIE tiny的subword切分模型路径;
* `word_dict_path`: 传入 ERNIE tiny的词语切分模型路径;

<center> <img width="600px" src="https://ai-studio-static-online.cdn.bcebos.com/c1a201c80acc46708cedc1c73614b0531b716c0299474859bf423ce0a5160170" /> </center>

In [4]:
dataset = Baidu()
reader = hub.reader.MultiLabelClassifyReader(
    dataset=dataset,
    vocab_path=module.get_vocab_path(),
    max_seq_len=256)

[2020-03-26 19:31:54,862] [    INFO] - Dataset label map = {'宇宙中的地球': 0, '地球运动的基本形式': 1, '地球的内部圈层结构及特点': 2, '社会主义是中国人民的历史性选择': 3, '减数分裂的概念': 4, '基因的分离规律的实质及应用': 5, '组成细胞的化合物': 6, '基因工程的概念': 7, '人体水盐平衡调节': 8, '拉马克的进化学说': 9, '地球的外部圈层结构及特点': 10, '人口增长与人口问题': 11, '生产活动与地域联系': 12, '复等位基因': 13, '液泡的结构和功能': 14, '人口与城市': 15, '兴奋在神经纤维上的传导': 16, '郡县制': 17, '第三产业的兴起和“新经济”的出现': 18, '太阳对地球的影响': 19, '区域可持续发展': 20, '基因工程的原理及技术': 21, '社会主义市场经济的伦理要求': 22, '地球运动的地理意义': 23, '皇帝制度': 24, '遗传与进化': 25, '生物科学与社会': 26, '中央官制——三公九卿制': 27, '避孕的原理和方法': 28, '激素调节': 29, '近代史': 30, '生物工程技术': 31, '内环境的稳态': 32, '细胞有丝分裂不同时期的特点': 33, '清末民主革命风潮': 34, '高尔基体的结构和功能': 35, '伴性遗传': 36, '文艺的春天': 37, '神经调节和体液调节的比较': 38, '海峡两岸关系的发展': 39, '科学思维常识': 40, '遗传的细胞基础': 41, '劳动就业与守法经营': 42, '古代史': 43, '政治': 44, '农业区位因素': 45, '选官、用官制度的变化': 46, '生活中的法律常识': 47, '人工授精、试管婴儿等生殖技术': 48, '生物性污染': 49, '培养基与无菌技术': 50, '遗传的分子基础': 51, '内质网的结构和功能': 52, '基因的自由组合规律的实质及应用': 53, '生态系统的营养结构': 54, '“重农抑商”政策': 55, '免疫系统的功能': 56, '生命活动离不开细胞': 57, '蛋白质的合成': 5

# Step3：选择优化策略和运行配置

In [6]:
strategy = hub.AdamWeightDecayStrategy(
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_proportion=0.0,
    lr_scheduler="linear_decay",
)

config = hub.RunConfig(use_cuda=True, 
                        use_data_parallel=True, 
                        use_pyreader=True, 
                        log_interval=200,
                        eval_interval=200,
                        num_epoch=5, 
                        batch_size=32, 
                        strategy=strategy)

[2020-03-26 19:32:06,783] [    INFO] - Checkpoint dir: ckpt_20200326193206


**优化策略**

针对ERNIE与BERT类任务，PaddleHub封装了适合这一任务的迁移学习优化策略`AdamWeightDecayStrategy`

- `learning_rate`: Fine-tune过程中的最大学习率；
- `weight_decay`: 模型的正则项参数，默认0.01，如果模型有过拟合倾向，可适当调高这一参数；
- `warmup_proportion`: 如果warmup_proportion>0, 例如0.1, 则学习率会在前10%的steps中线性增长至最高值learning_rate；
- `lr_scheduler`: 有两种策略可选(1) `linear_decay`策略学习率会在最高点后以线性方式衰减; `noam_decay`策略学习率会在最高点以多项式形式衰减；

PaddleHub提供了许多优化策略，如`AdamWeightDecayStrategy`、`ULMFiTStrategy`、`DefaultFinetuneStrategy`等，详细信息参见[策略](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub-API:-Strategy)。
<center> <img width="900px" src="https://ai-studio-static-online.cdn.bcebos.com/e7d3b9df642d443a8bdab1af0f3de93c0fa111ac5d3c4c9c88b2aa12f92dba64" /> </center>

**运行配置**

`RunConfig` 主要控制Fine-tune的训练，包含以下可控制的参数:

- `log_interval`: 进度日志打印间隔，默认每10个step打印一次；
- `eval_interval`: 模型评估的间隔，默认每100个step评估一次验证集；
- `save_ckpt_interval`: 模型保存间隔，请根据任务大小配置，默认只保存验证集效果最好的模型和训练结束的模型；
- `use_cuda`: 是否使用GPU训练，默认为False；
- use_pyreader: 是否使用pyreader，默认False；
- use_data_parallel: 是否使用并行计算，默认False。打开该功能依赖nccl库；
- `checkpoint_dir`: 模型checkpoint保存路径, 若用户没有指定，程序会自动生成；
- `num_epoch`: Fine-tune的轮数；
- `batch_size`: 训练的批大小，如果使用GPU，请根据实际情况调整batch_size；
- `enable_memory_optim`: 是否使用内存优化， 默认为True；
- `strategy`: Fine-tune优化策略；






# Step4: 构建网络并创建分类迁移任务进行Fine-tune

有了合适的预训练模型和准备要迁移的数据集后，我们开始组建一个Task。

1. 获取module的上下文环境，包括输入和输出的变量，以及Paddle Program；
2. 从输出变量中找到用于情感分类的文本特征pooled_output；
3. 在pooled_output后面接入一个全连接层，生成Task；

`TextClassifierTask`的参数有：

> `data_reader`：读取数据的reader；
>
> `config`: 运行配置；
>
> `feature`：从预训练提取的特征；
>
> `feed_list`：program需要输入的变量；
>
> `num_classes`：数据集的类别数量；
> 
> `metric_choic`：任务评估指标，默认为"acc"。metrics_choices支持训练过程中同时评估多个指标，作为最佳模型的判断依据，例如["matthews", "acc"]，"matthews"将作为主指标，为最佳模型的判断依据；


<center> <img width="600px" src="https://user-images.githubusercontent.com/48793257/69508060-97c0c180-0f6f-11ea-9886-00300da42683.png" /> </center>


[MultiLabelClassifierTask源码](https://github.com/PaddlePaddle/PaddleHub/blob/45179f3d06ed32ae60a4d8a573829b354548c4b5/paddlehub/finetune/task/classifier_task.py)

默认的评估指标就是auc
`metrics_choices="default"`


In [7]:
pooled_output = outputs["pooled_output"]

# feed_list的Tensor顺序不可以调整
feed_list = [
    inputs["input_ids"].name,
    inputs["position_ids"].name,
    inputs["segment_ids"].name,
    inputs["input_mask"].name,
]

cls_task = hub.MultiLabelClassifierTask(
    data_reader=reader,
    feature=pooled_output,
    feed_list=feed_list,
    num_classes=dataset.num_labels,
    config=config)

cls_task.finetune_and_eval()

2020-03-26 19:32:13,357-WARNING: paddle.fluid.layers.py_reader() may be deprecated in the near future. Please use paddle.fluid.io.DataLoader.from_generator() instead.
[2020-03-26 19:32:14,226] [    INFO] - Strategy with scheduler: {'warmup': 0.0, 'linear_decay': {'start_point': 0.0, 'end_learning_rate': 0}, 'noam_decay': False, 'discriminative': {'blocks': 0, 'factor': 2.6}, 'gradual_unfreeze': 0, 'slanted_triangle': {'cut_fraction': 0.0, 'ratio': 32}}, regularization: {'L2': 0.0, 'L2SP': 0.0, 'weight_decay': 0.01} and clip: {'GlobalNorm': 1.0, 'Norm': 0.0}
[2020-03-26 19:38:35,221] [    INFO] - Try loading checkpoint from ckpt_20200326193206/ckpt.meta
[2020-03-26 19:38:35,222] [    INFO] - PaddleHub model checkpoint not found, start from scratch...
[2020-03-26 19:38:35,422] [    INFO] - PaddleHub finetune start
[2020-03-26 19:41:14,034] [   TRAIN] - step 200 / 2981: loss=6.61374 auc=0.63222 auc_宇宙中的地球=0.79229 auc_地球运动的基本形式=0.64303 auc_地球的内部圈层结构及特点=0.53147 auc_社会主义是中国人民的历史性选择=0.55684 a

# 预测
当Finetune完成后，我们使用模型来进行预测，整个预测流程大致可以分为以下几步：
1. 构建网络
2. 生成预测数据的Reader
3. 切换到预测的Program
4. 加载预训练好的参数
5. 运行Program进行预测

<center> <img width="1024px" src="https://ai-studio-static-online.cdn.bcebos.com/320a7c13fcb044d3b45cb4720012bd26dbc10c283aa747dab43f72d1bad18643" /> </center>

> 预测代码如下：

In [11]:
import numpy as np

# index 转回 标签
inv_label_map = {val: key for key, val in reader.label_map.items()}
# 预测数据
data = [[d.text_a, d.text_b] for d in dataset.get_test_examples()]
# 预测标签
test_y = np.array([d.label for d in dataset.get_test_examples()])
# 预测
run_states = cls_task.predict(data)
# 把结果提取出来
results = [run_state.run_results for run_state in run_states]

[2020-03-27 14:10:08,479] [    INFO] - The best model has been loaded
[2020-03-27 14:10:08,480] [    INFO] - PaddleHub predict start
[2020-03-27 14:10:44,094] [    INFO] - PaddleHub predict finished.


In [12]:
test_y_pred = np.concatenate([np.argmax(batch_result, axis=2).T for batch_result in results])
test_y_pred.shape, test_y.shape

((5963, 97), (5963, 97))

In [13]:
from sklearn.metrics import f1_score, precision_score, recall_score
f1_micro = f1_score(test_y, test_y_pred, average='micro')
f1_macro = f1_score(test_y, test_y_pred, average='macro')
f1_samples = f1_score(test_y, test_y_pred, average='samples')
precision = precision_score(test_y, test_y_pred, average='micro')
recall = recall_score(test_y, test_y_pred, average='micro')

print('f1 micro:{}'.format(f1_micro))
print('f1 samples:{}'.format(f1_macro))
print('f1 macro:{}'.format(f1_samples))
print('precision:{}'.format(precision))
print('recall:{}'.format(recall))

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


f1 micro:0.9102579532234123
f1 samples:0.8318882086938779
f1 macro:0.9065945223360182
precision:0.9257445940432476
recall:0.8952809343434344


In [ ]:
res = [inv_label_map[idx] for idx in np.nonzero(test_y_pred[0])[0]]
res

['社会主义市场经济的伦理要求', '政治', '公民道德与伦理常识']

In [ ]:
pred =[[inv_label_map[idx] for idx in np.nonzero(line)[0]] for line in test_y_pred]
pred[3:5]

[['减数分裂的概念',
  '生物科学与社会',
  '避孕的原理和方法',
  '人工授精、试管婴儿等生殖技术',
  '生物性污染',
  '生物',
  '减数分裂与有丝分裂的比较'],
 ['近代史', '清末民主革命风潮', '历史']]

In [ ]:
real =[[inv_label_map[idx] for idx in np.nonzero(line)[0]] for line in test_y]
real[3:5]

[['生物科学与社会', '避孕的原理和方法', '人工授精、试管婴儿等生殖技术', '生物性污染', '生物', '减数分裂与有丝分裂的比较'],
 ['近代史', '清末民主革命风潮', '历史']]

# 将刚刚的整个过程封装

In [7]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score
def train(model_name="ernie_tiny", learning_rate=3e-5, max_seq_len=256, batch_size=32, epochs=5):
    module = hub.Module(name=model_name)
    inputs, outputs, program = module.context(trainable=True, max_seq_len=max_seq_len)

    dataset = Baidu()
    reader = hub.reader.MultiLabelClassifyReader(
        dataset=dataset,
        vocab_path=module.get_vocab_path(),
        max_seq_len=max_seq_len)

    strategy = hub.AdamWeightDecayStrategy(
        learning_rate=learning_rate,
        weight_decay=0.01,
        warmup_proportion=0.0,
        lr_scheduler="linear_decay",
    )

    config = hub.RunConfig(use_cuda=True, 
                            use_data_parallel=True, 
                            use_pyreader=True, 
                            log_interval=200,
                            eval_interval=200,
                            num_epoch=epochs, 
                            batch_size=batch_size, 
                            strategy=strategy)

    pooled_output = outputs["pooled_output"]

    # feed_list的Tensor顺序不可以调整
    feed_list = [
        inputs["input_ids"].name,
        inputs["position_ids"].name,
        inputs["segment_ids"].name,
        inputs["input_mask"].name,
    ]

    cls_task = hub.MultiLabelClassifierTask(
        data_reader=reader,
        feature=pooled_output,
        feed_list=feed_list,
        num_classes=dataset.num_labels,
        config=config)

    cls_task.finetune_and_eval()

    # index 转回 标签
    inv_label_map = {val: key for key, val in reader.label_map.items()}
    # 预测数据
    data = [[d.text_a, d.text_b] for d in dataset.get_test_examples()]
    # 预测标签
    test_y = np.array([d.label for d in dataset.get_test_examples()])
    # 预测
    run_states = cls_task.predict(data)
    # 把结果提取出来
    results = [run_state.run_results for run_state in run_states]

    test_y_pred = np.concatenate([np.argmax(batch_result, axis=2).T for batch_result in results])
        
    f1_micro = f1_score(test_y, test_y_pred, average='micro')
    f1_macro = f1_score(test_y, test_y_pred, average='macro')
    f1_samples = f1_score(test_y, test_y_pred, average='samples')
    precision = precision_score(test_y, test_y_pred, average='micro')
    recall = recall_score(test_y, test_y_pred, average='micro')

    print('f1 micro:{}'.format(f1_micro))
    print('f1 samples:{}'.format(f1_macro))
    print('f1 macro:{}'.format(f1_samples))
    print('precision:{}'.format(precision))
    print('recall:{}'.format(recall))
    
    return module, dataset, reader, cls_task, (f1_micro, f1_macro, f1_samples)

In [8]:
%cd /home/aistudio/
!mkdir -p .paddlehub/cache

/home/aistudio


In [9]:
module, dataset, reader, cls_task, res = train(model_name="bert_wwm_ext_chinese_L-12_H-768_A-12", epochs=5)

[2020-03-27 13:03:44,039] [    INFO] - Installing bert_wwm_ext_chinese_L-12_H-768_A-12 module
[2020-03-27 13:03:44,068] [    INFO] - Module bert_wwm_ext_chinese_L-12_H-768_A-12 already installed in /home/aistudio/.paddlehub/modules/bert_wwm_ext_chinese_L-12_H-768_A-12
[2020-03-27 13:03:44,687] [    INFO] - Set maximum sequence length of input tensor to 256
[2020-03-27 13:03:44,694] [    INFO] - The shape of input tensor[input_ids] set to [-1, 256, 1]
[2020-03-27 13:03:44,695] [    INFO] - The shape of input tensor[position_ids] set to [-1, 256, 1]
[2020-03-27 13:03:44,695] [    INFO] - The shape of input tensor[segment_ids] set to [-1, 256, 1]
[2020-03-27 13:03:44,696] [    INFO] - The shape of input tensor[input_mask] set to [-1, 256, 1]
[2020-03-27 13:03:44,697] [    INFO] - 199 pretrained paramaters loaded by PaddleHub
[2020-03-27 13:03:47,513] [    INFO] - Dataset label map = {'宇宙中的地球': 0, '地球运动的基本形式': 1, '地球的内部圈层结构及特点': 2, '社会主义是中国人民的历史性选择': 3, '减数分裂的概念': 4, '基因的分离规律的实质及应用': 5, '组

f1 micro:0.9102579532234123
f1 samples:0.8318882086938779
f1 macro:0.9065945223360182


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
